This noteboook explores how to provide a composite score from unsupervised models that output a percentile rank and confidence, and where a compositor object can ascribe importances to each model.

# Example model outputs for a score value and confidence.

In [ ]:
!pip install -r requirements.txt

In [1]:
import sys
from ipywidgets import FloatSlider, GridspecLayout, HTML
import numpy as np

def calculate_score(models):
    
    # We combine the confidence and importance from a model into one score
    for model in models:
        model['confidence_importance'] = np.sqrt(model['confidence'] * model['importance'])
        
    # Then normalize by that combined confidence and importance score
    total = 0
    for model in models:
        total += model['confidence_importance']
        
    for model in models:
        model['relative_weight'] = model['confidence_importance'] / total
        model['contribution'] = model['relative_weight'] * model['value']
        
    composite_score = sum(model['contribution'] for model in models)
    
    return composite_score

# Outputs from individual models

Each model outputs:

  * A 0-1 quantile value
  * A (pseudo-)confidence score.
  
The umbrella model imposes:

  * An importance score.
  
The confidence and importance values for each model get conflated via:

  $x_{i} = \sqrt{confidence_{i} \cdot importance_{i}}$
  
Then we sum these values so that we can normalize.

  $X = \Sigma_{i} x_{i}$
  
Then, we normalize the conflated term to get a relative weight for the model:

  $w_{i} = x_{i} / X$
  
And set the contribution to:

  $m_{i} = w_{i} \cdot quantile\_score_{i}$
  
And get the composite score:

  $\Sigma_{i} m_{i}$

In [2]:
models = []
models.append(dict(value=.5, confidence=1.))
models.append(dict(value=.5, confidence=.01))
models.append(dict(value=.5, confidence=.5))
for model in models:
    model['importance'] = 1/len(models)
models

[{'value': 0.5, 'confidence': 1.0, 'importance': 0.3333333333333333},
 {'value': 0.5, 'confidence': 0.01, 'importance': 0.3333333333333333},
 {'value': 0.5, 'confidence': 0.5, 'importance': 0.3333333333333333}]

In [3]:
calculate_score(models)

0.5

# Playground

In [4]:
n_models = len(models)
value_sliders = [FloatSlider(value=m['value'], min=.0, max=1., step=.01) for m in models]
confidence_sliders = [FloatSlider(value=m['confidence'], min=.0, max=1., step=.01) for m in models]
importance_sliders = [FloatSlider(value=m['importance'], min=0.01, max=1., step=.01) for m in models]
    
def handle_value_slider_change(change):
    global value_sliders, models
    for slider, model in zip(value_sliders, models):
        if change['owner'] == slider:
            model['value'] = change['new']
            break
    sys.stdout.write(f'Composite score: {calculate_score(models):.3f}\r')
    
def handle_confidence_slider_change(change):
    global confidence_sliders, models
    for slider, model in zip(confidence_sliders, models):
        if change['owner'] == slider:
            model['confidence'] = change['new']
            break
    sys.stdout.write(f'Composite score: {calculate_score(models):.3f}\r')

def handle_importance_slider_change(change):
    global importance_sliders, models
    
    total = 0
    for slider in importance_sliders:
        if slider != change['owner']:
            total += slider.value
        slider.unobserve(handle_importance_slider_change, names='value')
    
    if total == 0:
        total = .01
    for slider in importance_sliders:
        if slider != change['owner']:
            slider.value *= (1 - change['new']) / total
            
    for model, slider in zip(models, importance_sliders):
        slider.observe(handle_importance_slider_change, names='value')
        model['importance'] = slider.value
    
    sys.stdout.write(f'Composite score: {calculate_score(models):.3f}\r')
    
for slider in value_sliders:
    slider.observe(handle_value_slider_change, names='value')
for slider in confidence_sliders:
    slider.observe(handle_confidence_slider_change, names='value')
for slider in importance_sliders:
    slider.observe(handle_importance_slider_change, names='value')

grid = GridspecLayout(n_models+1, 3)

header = HTML("<h4>Value</h4>")
grid[0, 0] = header
for i, slider in enumerate(value_sliders):
    grid[i+1, 0] = slider
    
header = HTML("<h4>Confidence</h4>")
grid[0, 1] = header
for i, slider in enumerate(confidence_sliders):
    grid[i+1, 1] = slider

header = HTML("<h4>Importance (sum to 1)</h4>")
grid[0, 2] = header
for i, slider in enumerate(importance_sliders):
    grid[i+1, 2] = slider

grid

GridspecLayout(children=(HTML(value='<h4>Value</h4>', layout=Layout(grid_area='widget001')), FloatSlider(value…